# Pandas json_normalize()

In [1]:
import pandas as pd

## 1. Flattening a simple JSON

**Single Dictionary**

In [2]:
a_dict = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
}

df = pd.json_normalize(a_dict)

In [3]:
df

,school,location,ranking
0,ABC primary school,London,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   school    1 non-null      object
 1   location  1 non-null      object
 2   ranking   1 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 152.0+ bytes


**List of Dictionaries**

In [5]:
json_list = [
    { 'class': 'Year 1', 'student number': 20, 'room': 'Yellow' },
    { 'class': 'Year 2', 'student number': 25},
]

pd.json_normalize(json_list)

,class,student number,room
0,Year 1,20,Yellow
1,Year 2,25,NaN


## 2. Flattening multiple levels JSON

**Simple Multiple Level JSON**

In [6]:
json_obj2 = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
    'info': {
        'president': 'John Kasich',
        'contacts': {
          'email': {
              'admission': 'admission@abc.com',
              'general': 'info@abc.com'
          },
          'tel': '123456789',
      }
    }
}

In [7]:
pd.json_normalize(json_obj2)

,school,location,ranking,info.president,info.contacts.email.admission,info.contacts.email.general,info.contacts.tel
0,ABC primary school,London,2,John Kasich,admission@abc.com,info@abc.com,123456789


using **max_level** to determine the dict depth

In [8]:
pd.json_normalize(json_obj2, max_level=2)

,school,location,ranking,info.president,info.contacts.email,info.contacts.tel
0,ABC primary school,London,2,John Kasich,"{'admission': 'admission@abc.com', 'general': ...",123456789


**List of Multiple Level Dictionaries**

In [9]:
json_list2 = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask' 
            }
        }
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'math': 'Alan Turing', 
                'physics': 'Albert Einstein' 
            }
        }
    },
]

In [10]:
pd.json_normalize(json_list2)

,class,student count,room,info.teachers.math,info.teachers.physics
0,Year 1,20,Yellow,Rick Scott,Elon Mask
1,Year 2,25,Blue,Alan Turing,Albert Einstein


In [11]:
pd.json_normalize(json_list2, max_level=1)

,class,student count,room,info.teachers
0,Year 1,20,Yellow,"{'math': 'Rick Scott', 'physics': 'Elon Mask'}"
1,Year 2,25,Blue,"{'math': 'Alan Turing', 'physics': 'Albert Ein..."


# 3. Flattening JSON with a nested list

**Simple Nested List**

In [12]:
json_obj3 = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
    'info': {
        'president': 'John Kasich',
        'contacts': {
          'email': {
              'admission': 'admission@abc.com',
              'general': 'info@abc.com'
          },
          'tel': '123456789',
      }
    },
    'students': [
      { 'name': 'Tom' },
      { 'name': 'James' },
      { 'name': 'Jacqueline' }
    ],
}

In [13]:
pd.json_normalize(json_obj3)

,school,location,ranking,students,info.president,info.contacts.email.admission,info.contacts.email.general,info.contacts.tel
0,ABC primary school,London,2,"[{'name': 'Tom'}, {'name': 'James'}, {'name': ...",John Kasich,admission@abc.com,info@abc.com,123456789


use **record_path** to flatten the nested list

In [14]:
pd.json_normalize(json_obj3, record_path = ['students'])

,name
0,Tom
1,James
2,Jacqueline


use **meta** to specify a list of metadata we want to include in nested list

In [15]:
pd.json_normalize(
        json_obj3, 
        record_path = ['students'],
        meta = ['school', ['info', 'contacts','tel']]
    )

,name,school,info.contacts.tel
0,Tom,ABC primary school,123456789
1,James,ABC primary school,123456789
2,Jacqueline,ABC primary school,123456789


**List of Dictionaries**

In [16]:
json_list3 = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask' 
            }
        },
        'students': [
            { 
                'name': 'Tom', 
                'sex': 'M', 
                'grades': { 'math': 66, 'physics': 77 } 
            },
            { 
                'name': 'James', 
                'sex': 'M', 
                'grades': { 'math': 80, 'physics': 78 } 
            },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'math': 'Alan Turing', 
                'physics': 'Albert Einstein' 
            }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

In [17]:
pd.json_normalize(json_list3)

,class,student count,room,students,info.teachers.math,info.teachers.physics
0,Year 1,20,Yellow,"[{'name': 'Tom', 'sex': 'M', 'grades': {'math'...",Rick Scott,Elon Mask
1,Year 2,25,Blue,"[{'name': 'Tony', 'sex': 'M'}, {'name': 'Jacqu...",Alan Turing,Albert Einstein


In [18]:
pd.json_normalize(json_list3, record_path = ['students'])

,name,sex,grades.math,grades.physics
0,Tom,M,66.0,77.0
1,James,M,80.0,78.0
2,Tony,M,NaN,NaN
3,Jacqueline,F,NaN,NaN


In [19]:
pd.json_normalize(json_list3, 
                record_path = ['students'],
                meta = ['class', 'room', ['info', 'teachers', 'math']]
                 )

,name,sex,grades.math,grades.physics,class,room,info.teachers.math
0,Tom,M,66.0,77.0,Year 1,Yellow,Rick Scott
1,James,M,80.0,78.0,Year 1,Yellow,Rick Scott
2,Tony,M,NaN,NaN,Year 2,Blue,Alan Turing
3,Jacqueline,F,NaN,NaN,Year 2,Blue,Alan Turing


# 4. The `error` argument

In [20]:
data = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask',
            }
        },
        'students': [
            { 'name': 'Tom', 'sex': 'M' },
            { 'name': 'James', 'sex': 'M' },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                 # no math teacher
                 'physics': 'Albert Einstein'
            }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

user **errors** = `'ignore'` to ignore all missing values

In [21]:
pd.json_normalize(
    data,
    record_path = 'students',
    meta = ['class', 'room', ['info', 'teachers', 'math'], ['info', 'teachers', 'physics']],
    errors = 'ignore'
    )

,name,sex,class,room,info.teachers.math,info.teachers.physics
0,Tom,M,Year 1,Yellow,Rick Scott,Elon Mask
1,James,M,Year 1,Yellow,Rick Scott,Elon Mask
2,Tony,M,Year 2,Blue,NaN,Albert Einstein
3,Jacqueline,F,Year 2,Blue,NaN,Albert Einstein


# 5. Custom seperator using the `sep` argument

In [22]:
pd.json_normalize(
    data,
    record_path = ['students'],
    meta = ['class', 'room', ['info', 'teachers', 'math']],
    errors = 'ignore',
    sep = ' -> '
    )

,name,sex,class,room,info -> teachers -> math
0,Tom,M,Year 1,Yellow,Rick Scott
1,James,M,Year 1,Yellow,Rick Scott
2,Tony,M,Year 2,Blue,NaN
3,Jacqueline,F,Year 2,Blue,NaN


# 6. Add prefix for meta and record data

In [23]:
pd.json_normalize(
    data,
    record_path = ['students'],
    meta = ['class'],
    meta_prefix = 'meta-',
    record_prefix = 'student-'
    )

,student-name,student-sex,meta-class
0,Tom,M,Year 1
1,James,M,Year 1
2,Tony,M,Year 2
3,Jacqueline,F,Year 2


# 7. Working with a local file

In [30]:
import json

In [31]:
with open('example.json') as file:
    data = json.loads(file.read())

In [32]:
pd.json_normalize(data)

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78


# 8. Working with URL

In [34]:
import requests

In [35]:
url = 'http://raw.githubusercontent.com/BindiChen/machine-learning/master/data-analysis/027-pandas-convert-json/data/simple.json'
json_data = requests.get(url)
data = json_data.json()

In [36]:
pd.json_normalize(data)

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78
